In [ ]:
import time
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ----- SCRAPING THE DATA -----

def wait_for_page_to_load(driver, wait):
	title = driver.title
	try:
		wait.until(
			lambda d: d.execute_script("return document.readyState") == "complete"
		)
	except:
		print(f"The webpage \"{title}\" did not get fully laoded.\n")
	else:
		print(f"The webpage \"{title}\" did get fully laoded.\n")


# options
chrome_options = Options()
chrome_options.add_argument("--disable-http2")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("--enable-features=NetworkServiceInProcess")
chrome_options.add_argument("--disable-features=NetworkService")
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36"
)

driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()

# explicit wait
wait = WebDriverWait(driver, 5)

# accessing the target webpage
url = "https://www.99acres.com/"
driver.get(url)
wait_for_page_to_load(driver, wait)

# identify and enter text into search bar
try:
	search_bar = wait.until(
		EC.presence_of_element_located((By.XPATH, '//*[@id="keyword2"]'))
	)
except:
	print("Timeout while locating Search Bar.\n")
else:
	search_bar.send_keys("Bangalore")
	time.sleep(2)

# selecting valid option from list
try:
	valid_option = wait.until(
		EC.element_to_be_clickable((By.XPATH, '//*[@id="0"]'))
	)
except:
	print("Timeout while locating valid search option.\n")
else:
	valid_option.click()
	time.sleep(2)

# click on Search button
try:
	search_button = wait.until(
		EC.element_to_be_clickable((By.XPATH, '//*[@id="searchform_search_btn"]'))
	)
except:
	print("Timeout while clicking on \"Search\" button.\n")
else:
	search_button.click()
	wait_for_page_to_load(driver, wait)

# adjust the Budget slider
try:
	slider = wait.until(
		EC.element_to_be_clickable((By.XPATH, '//*[@id="budgetLeftFilter_max_node"]'))
	)
except:
	print("Timeout while clicking on Budget slider circle.\n")
else:
	actions = ActionChains(driver)
	(
		actions
		.click_and_hold(slider)
		.move_by_offset(-73, 0)
		.release()
		.perform()
	)
	time.sleep(2)

# filter results to show genuine listings
# 1. Verified
verified = wait.until(
	EC.element_to_be_clickable((By.XPATH, '/html[1]/body[1]/div[1]/div[1]/div[1]/div[4]/div[3]/div[1]/div[3]/section[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[3]/span[2]'))
)
verified.click()
time.sleep(1)

# 2. Ready To Move
ready_to_move = wait.until(
	EC.element_to_be_clickable((By.XPATH, '/html[1]/body[1]/div[1]/div[1]/div[1]/div[4]/div[3]/div[1]/div[3]/section[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[5]/span[2]'))
)
ready_to_move.click()
time.sleep(1)

# moving to the right side to unhide remaining filters
while True:
	try:
		filter_right_button = wait.until(
			EC.presence_of_element_located((By.XPATH, "//i[contains(@class,'iconS_Common_24 icon_upArrow cc__rightArrow')]"))
		)
	except:
		print("Timeout because we have uncovered all filters.\n")
		break
	else:
		filter_right_button.click()
		time.sleep(10)

# 3. With Photos
with_photos = wait.until(
	EC.element_to_be_clickable((By.XPATH, '/html[1]/body[1]/div[1]/div[1]/div[1]/div[4]/div[3]/div[1]/div[3]/section[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[1]/div[6]/span[2]'))
)
with_photos.click()
time.sleep(1)

# 4. With Videos
with_videos = wait.until(
	EC.element_to_be_clickable((By.XPATH, '/html[1]/body[1]/div[1]/div[1]/div[1]/div[4]/div[3]/div[1]/div[3]/section[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[1]/div[7]/span[2]'))
)
with_videos.click()
time.sleep(3)

# navigate pages and extract data
data = []
page_count = 0
while True:
	page_count += 1
	try:
		next_page_button = driver.find_element(By.XPATH, "//a[normalize-space()='Next Page >']")
	except:
		print(f"Timeout because we have navigated all the {page_count} pages.\n")
		break
	else:
		try:
			driver.execute_script("window.scrollBy(0, arguments[0].getBoundingClientRect().top - 100);", next_page_button)
			time.sleep(20)
	
			# scraping the data
			rows = driver.find_elements(By.CLASS_NAME, "tupleNew__TupleContent")
			for row in rows:
				# property name
				try:
					name = row.find_element(By.CLASS_NAME, "tupleNew__headingNrera").text
				except:
					name = np.nan

				# property location
				try:
					location = row.find_element(By.CLASS_NAME, "tupleNew__propType").text
				except:
					location = np.nan

				# property price
				try:
					price = row.find_element(By.CLASS_NAME, "tupleNew__priceValWrap").text
				except:
					price = np.nan

				# property area and bhk
				try:
					elements = row.find_elements(By.CLASS_NAME, "tupleNew__area1Type")
				except:
					area, bhk = [np.nan, np.nan]
				else:
					area, bhk = [ele.text for ele in elements]
					
				property = {
					"name": name,
					"location": location,
					"price": price,
					"area": area,
					"bhk": bhk
				}
				data.append(property)
			
			wait.until(
				EC.element_to_be_clickable((By.XPATH, "//a[normalize-space()='Next Page >']"))
			).click()
			time.sleep(10)
		except Exception as e:
			print(f'Exception: {e}')
			print("Timeout while clicking on \"Next Page\".\n")

# scraping data from the last page
rows = driver.find_elements(By.CLASS_NAME, "tupleNew__TupleContent")
for row in rows:
	# property name
	try:
		name = row.find_element(By.CLASS_NAME, "tupleNew__headingNrera").text
	except:
		name = np.nan

	# property location
	try:
		location = row.find_element(By.CLASS_NAME, "tupleNew__propType").text
	except:
		location = np.nan

	# property price
	try:
		price = row.find_element(By.CLASS_NAME, "tupleNew__priceValWrap").text
	except:
		price = np.nan

	# property area and bhk
	try:
		elements = row.find_elements(By.CLASS_NAME, "tupleNew__area1Type")
	except:
		area, bhk = [np.nan, np.nan]
	else:
		area, bhk = [ele.text for ele in elements]
					
	property = {
		"name": name,
		"location": location,
		"price": price,
		"area": area,
		"bhk": bhk
	}
	data.append(property)

time.sleep(2)
driver.quit()

# ----- CLEANING THE DATA -----

df_properties = (
	pd
	.DataFrame(data)
	.drop_duplicates()
	.apply(lambda col: col.str.strip().str.lower() if col.dtype == "object" else col)
	.assign(
		is_starred=lambda df_: df_.name.str.contains("\n").astype(int),
		name=lambda df_: (
			df_
			.name
			.str.replace("\n[0-9.]+", "", regex=True)
			.str.strip()
			.replace("adroit district s", "adroit district's")
		),
		location=lambda df_: (
			df_
			.location
			.str.replace("chennai", "")
			.str.strip()
			.str.replace(",$", "", regex=True)
			.str.split("in")
			.str[-1]
			.str.strip()
		),
		price=lambda df_: (
			df_
			.price
			.str.replace("₹", "")
			.apply(lambda val: float(val.replace("lac", "").strip()) if "lac" in val else float(val.replace("cr", "").strip()) * 100)
		),
		area=lambda df_: (
			df_
			.area
			.str.replace("sqft", "")
			.str.strip()
			.str.replace(",", "")
			.pipe(lambda ser: pd.to_numeric(ser))
		),
		bhk=lambda df_: (
			df_
			.bhk
			.str.replace("bhk", "")
			.str.strip()
			.pipe(lambda ser: pd.to_numeric(ser))
		)
	)
	.rename(columns={
		"price": "price_lakhs",
		"area": "area_sqft"
	})
	.reset_index(drop=True)
	.to_excel("chennai-properties-99acres.xlsx", index=False)
)

The webpage "India Real Estate Property Site - Buy Sell Rent Properties Portal - 99acres.com" did get fully laoded.

The webpage "Property in Bangalore - Real Estate in Bangalore" did get fully laoded.

Timeout because we have uncovered all filters.

Timeout while clicking on "Next Page".

Timeout because we have navigated all the 2 pages.



In [9]:
import time
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

class ProprertyScraper:
	def __init__(self, url, timeout=5):
		self.url = url
		self.data = []
		self.driver = self._initialize_driver()
		self.wait = WebDriverWait(self.driver, timeout=timeout)


	def _initialize_driver(self):
		chrome_options = Options()
		chrome_options.add_argument("--disable-http2")
		chrome_options.add_argument("--incognito")
		chrome_options.add_argument("--disable-blink-features=AutomationControlled")
		chrome_options.add_argument("--ignore-certificate-errors")
		chrome_options.add_argument("--enable-features=NetworkServiceInProcess")
		chrome_options.add_argument("--disable-features=NetworkService")
		chrome_options.add_argument(
		    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36"
		)
		driver = webdriver.Chrome(options=chrome_options)
		driver.maximize_window()
		return driver


	def _wait_for_page_to_load(self):
		title = self.driver.title
		try:
			self.wait.until(
				lambda d: d.execute_script("return document.readyState") == "complete"
			)
		except:
			print(f"The webpage \"{title}\" did not get fully laoded.\n")
		else:
			print(f"The webpage \"{title}\" did get fully laoded.\n")

	
	def access_website(self):
		self.driver.get(self.url)
		self._wait_for_page_to_load()


	def search_properties(self, text):
		# locating and entering text in search bar
		try:
			search_bar = self.wait.until(
				EC.presence_of_element_located((By.XPATH, '//*[@id="keyword2"]'))
			)
		except:
			print("Timeout while locating Search Bar.\n")
		else:
			search_bar.send_keys(text)
			time.sleep(2)
		
		# selecting valid option from list
		try:
			valid_option = self.wait.until(
				EC.element_to_be_clickable((By.XPATH, '//*[@id="0"]'))
			)
		except:
			print("Timeout while locating valid search option.\n")
		else:
			valid_option.click()
			time.sleep(2)
		
		# click on Search button
		try:
			search_button = self.wait.until(
				EC.element_to_be_clickable((By.XPATH, '//*[@id="searchform_search_btn"]'))
			)
		except:
			print("Timeout while clicking on \"Search\" button.\n")
		else:
			search_button.click()
			self._wait_for_page_to_load()


	def adjust_budget_slider(self, offset):
		try:
			slider = self.wait.until(
				EC.element_to_be_clickable((By.XPATH, '//*[@id="budgetLeftFilter_max_node"]'))
			)
		except:
			print("Timeout while clicking on Budget slider circle.\n")
		else:
			actions = ActionChains(self.driver)
			(
				actions
				.click_and_hold(slider)
				.move_by_offset(offset, 0)
				.release()
				.perform()
			)
			time.sleep(2)


	def apply_filters(self):
		# 1. Verified
		verified = self.wait.until(
			EC.element_to_be_clickable((By.XPATH, '/html[1]/body[1]/div[1]/div[1]/div[1]/div[4]/div[3]/div[1]/div[3]/section[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[3]/span[2]'))
		)
		verified.click()
		time.sleep(1)
		
		# 2. Ready To Move
		ready_to_move = self.wait.until(
			EC.element_to_be_clickable((By.XPATH, '/html[1]/body[1]/div[1]/div[1]/div[1]/div[4]/div[3]/div[1]/div[3]/section[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[5]/span[2]'))
		)
		ready_to_move.click()
		time.sleep(1)
		
		# moving to the right side to unhide remaining filters
		while True:
			try:
				filter_right_button = self.wait.until(
					EC.presence_of_element_located((By.XPATH, "//i[contains(@class,'iconS_Common_24 icon_upArrow cc__rightArrow')]"))
				)
			except:
				print("Timeout because we have uncovered all filters.\n")
				break
			else:
				filter_right_button.click()
				time.sleep(10)
		
		# 3. With Photos
		with_photos = self.wait.until(
			EC.element_to_be_clickable((By.XPATH, '/html[1]/body[1]/div[1]/div[1]/div[1]/div[4]/div[3]/div[1]/div[3]/section[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[1]/div[6]/span[2]'))
		)
		with_photos.click()
		time.sleep(10)
		
		# 4. With Videos
		with_videos = self.wait.until(
			EC.element_to_be_clickable((By.XPATH, '/html[1]/body[1]/div[1]/div[1]/div[1]/div[4]/div[3]/div[1]/div[3]/section[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[1]/div[7]/span[2]'))
		)
		with_videos.click()
		time.sleep(10)


	def _extract_data(self, row, by, value):
		try:
			return row.find_element(by, value).text
		except:
			return np.nan
	

	def scrape_webpage(self):
		rows = self.driver.find_elements(By.CLASS_NAME, "tupleNew__TupleContent")
		for row in rows:
			property = {
				"name": self._extract_data(row, By.CLASS_NAME, "tupleNew__headingNrera"),
				"location": self._extract_data(row, By.CLASS_NAME, "tupleNew__propType"),
				"price": self._extract_data(row, By.CLASS_NAME, "tupleNew__priceValWrap")
			}
		
			try:
				elements = row.find_elements(By.CLASS_NAME, "tupleNew__area1Type")
			except:
				property["area"], property["bhk"] = [np.nan, np.nan]
			else:
				property["area"], property["bhk"] = [ele.text for ele in elements]
				
			self.data.append(property)
		

	def navigate_pages_and_scrape_data(self):
		page_count = 0
		while True:
			page_count += 1
			try:
				self.scrape_webpage()
				next_page_button = self.driver.find_element(By.XPATH, "//a[normalize-space()='Next Page >']")
			except:
				print(f"We have scraped {page_count} pages.\n")
				break
			else:
				try:
					self.driver.execute_script("window.scrollBy(0, arguments[0].getBoundingClientRect().top - 100);", next_page_button)
					time.sleep(2)			
					self.wait.until(
						EC.element_to_be_clickable((By.XPATH, "//a[normalize-space()='Next Page >']"))
					).click()
					time.sleep(10)
				except:
					print("Timeout while clicking on \"Next Page\".\n")


	def clean_data_and_save_as_excel(self, file_name):
		df_properties = (
			pd
			.DataFrame(self.data)
			.drop_duplicates()
			.apply(lambda col: col.str.strip().str.lower() if col.dtype == "object" else col)
			.assign(
				is_starred=lambda df_: df_.name.str.contains("\n").astype(int),
				name=lambda df_: (
					df_
					.name
					.str.replace("\n[0-9.]+", "", regex=True)
					.str.strip()
					.replace("adroit district s", "adroit district's")
				),
				location=lambda df_: (
					df_
					.location
					.str.replace("chennai", "")
					.str.strip()
					.str.replace(",$", "", regex=True)
					.str.split("in")
					.str[-1]
					.str.strip()
				),
				price=lambda df_: (
					df_
					.price
					.str.replace("₹", "")
					.apply(lambda val: float(val.replace("lac", "").strip()) if "lac" in val else float(val.replace("cr", "").strip()) * 100)
				),
				area=lambda df_: (
					df_
					.area
					.str.replace("sqft", "")
					.str.strip()
					.str.replace(",", "")
					.pipe(lambda ser: pd.to_numeric(ser))
				),
				bhk=lambda df_: (
					df_
					.bhk
					.str.replace("bhk", "")
					.str.strip()
					.pipe(lambda ser: pd.to_numeric(ser))
				)
			)
			.rename(columns={
				"price": "price_lakhs",
				"area": "area_sqft"
			})
			.reset_index(drop=True)
		)
		df_properties.to_excel(f"{file_name}.xlsx", index=False)

	
	def run(self, text="Chennai", offset=-100, file_name="properties"):
		try:
			self.access_website()
			self.search_properties(text)
			self.adjust_budget_slider(offset)
			self.apply_filters()
			self.navigate_pages_and_scrape_data()
			self.clean_data_and_save_as_excel(file_name)
		finally:
			time.sleep(5)
			self.driver.quit()


if __name__ == "__main__":
	scraper = ProprertyScraper(url="https://www.99acres.com/")
	scraper.run(
		text="chennai",
		offset=-73,
		file_name="chennai-properties"
	)

The webpage "India Real Estate Property Site - Buy Sell Rent Properties Portal - 99acres.com" did get fully laoded.

The webpage "Property in Chennai - Real Estate in Chennai" did get fully laoded.

Timeout because we have uncovered all filters.

Timeout while clicking on "Next Page".

We have scraped 2 pages.



In [10]:
import time
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# ----- SCRAPING THE DATA -----

def wait_for_page_to_load(driver, wait_time=10):
    """Wait for page to load completely"""
    title = driver.title
    wait = WebDriverWait(driver, wait_time)
    try:
        wait.until(lambda d: d.execute_script("return document.readyState") == "complete")
        print(f"The webpage \"{title}\" loaded successfully.\n")
        # Additional wait for any AJAX content to load
        time.sleep(2)
    except:
        print(f"Warning: The webpage \"{title}\" may not have fully loaded.\n")

def scroll_to_element(driver, element):
    """Scroll to make element visible"""
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
    time.sleep(1)

def extract_property_data(row):
    """Extract data from a property row"""
    property_data = {}
    
    # Property name
    try:
        property_data["name"] = row.find_element(By.CSS_SELECTOR, ".tupleNew__headingNrera, .srpTuple__propertyName").text
    except:
        property_data["name"] = np.nan

    # Property location
    try:
        property_data["location"] = row.find_element(By.CSS_SELECTOR, ".tupleNew__propType, .srpTuple__locationName").text
    except:
        property_data["location"] = np.nan

    # Property price
    try:
        property_data["price"] = row.find_element(By.CSS_SELECTOR, ".tupleNew__priceValWrap, .srpTuple__price").text
    except:
        property_data["price"] = np.nan

    # Property area and bhk
    try:
        elements = row.find_elements(By.CSS_SELECTOR, ".tupleNew__area1Type, .srpTuple__dFlex.srpTuple__srpDataWrap span:nth-child(1), .srpTuple__dFlex.srpTuple__srpDataWrap span:nth-child(3)")
        if len(elements) >= 2:
            property_data["area"] = elements[0].text
            property_data["bhk"] = elements[1].text
        else:
            property_data["area"] = elements[0].text if elements else np.nan
            property_data["bhk"] = np.nan
    except:
        property_data["area"] = np.nan
        property_data["bhk"] = np.nan
        
    return property_data

# Configure Chrome options
chrome_options = Options()
chrome_options.add_argument("--disable-http2")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("--disable-notifications")  # Disable notifications
chrome_options.add_argument("--disable-popup-blocking")  # Disable popup blocks
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.112 Safari/537.36"
)

# Initialize driver
driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()

# Create WebDriverWait instance with increased timeout
wait = WebDriverWait(driver, 15)  # Increased timeout to 15 seconds

# Track our data
data = []
page_count = 0
total_properties = 0

try:
    # Access the target webpage
    url = "https://www.99acres.com/"
    driver.get(url)
    wait_for_page_to_load(driver)

    # Search for Chennai properties
    try:
        search_bar = wait.until(EC.presence_of_element_located((By.ID, "keyword2")))
        search_bar.clear()
        search_bar.send_keys("Chennai")
        time.sleep(2)
        
        # Select the first suggestion
        try:
            valid_option = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='autocomplete_drop']//li[1]")))
            valid_option.click()
        except:
            print("Could not click the first search suggestion. Trying alternative approach.")
            search_bar.send_keys(Keys.DOWN)
            search_bar.send_keys(Keys.ENTER)
        
        time.sleep(1)
        
        # Click search button
        search_button = wait.until(EC.element_to_be_clickable((By.ID, "searchform_search_btn")))
        search_button.click()
        wait_for_page_to_load(driver)
        
        print("Successfully navigated to Chennai properties page.")
    except Exception as e:
        print(f"Error during search process: {e}")
    
    # Apply filters more robustly
    try:
        # Set max budget using slider
        try:
            slider = wait.until(EC.presence_of_element_located((By.XPATH, '//*[contains(@id,"budgetLeftFilter_max_node")]')))
            scroll_to_element(driver, slider)
            actions = ActionChains(driver)
            actions.click_and_hold(slider).move_by_offset(-73, 0).release().perform()
            time.sleep(2)
            print("Budget filter applied successfully.")
        except Exception as e:
            print(f"Budget slider adjustment failed: {e}")
        
        # Apply Verified, Ready To Move, With Photos filters
        filter_options = [
            {"name": "Verified", "xpath": "//div[contains(@class,'srpFilterWrapper')]//span[contains(text(),'Verified')]/.."},
            {"name": "Ready To Move", "xpath": "//div[contains(@class,'srpFilterWrapper')]//span[contains(text(),'Ready To Move')]/.."},
            {"name": "With Photos", "xpath": "//div[contains(@class,'srpFilterWrapper')]//span[contains(text(),'With Photos')]/.."}
        ]
        
        for filter_opt in filter_options:
            try:
                # Try to find the filter button
                filter_element = driver.find_element(By.XPATH, filter_opt["xpath"])
                scroll_to_element(driver, filter_element)
                filter_element.click()
                print(f"Applied filter: {filter_opt['name']}")
                time.sleep(2)
            except:
                print(f"Could not apply filter: {filter_opt['name']}")
                
                # Try to navigate right to see more filters if needed
                try:
                    right_arrow = driver.find_element(By.XPATH, "//i[contains(@class,'rightArrow')]")
                    scroll_to_element(driver, right_arrow)
                    right_arrow.click()
                    time.sleep(2)
                    
                    # Try again to apply the filter
                    filter_element = driver.find_element(By.XPATH, filter_opt["xpath"])
                    scroll_to_element(driver, filter_element)
                    filter_element.click()
                    print(f"Applied filter: {filter_opt['name']} after scrolling")
                    time.sleep(2)
                except:
                    print(f"Still couldn't apply filter: {filter_opt['name']}")
        
        # Wait for filters to apply and results to load
        time.sleep(5)
        wait_for_page_to_load(driver)
        print("All filters applied, proceeding to scrape data.")
        
    except Exception as e:
        print(f"Error applying filters: {e}")
    
    # Process all pages
    while True:
        page_count += 1
        print(f"Processing page {page_count}...")
        
        # Wait for property listings to load
        try:
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".srpTuple__tupleDetails, .tupleNew__TupleContent")))
        except TimeoutException:
            print("Could not find property listings on this page.")
        
        # Scroll down the page to ensure all content loads
        driver.execute_script("window.scrollTo(0, 0);")  # Start at top
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollBy(0, 500);")
            time.sleep(0.5)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        
        # Extract data from all property listings on current page
        property_rows = driver.find_elements(By.CSS_SELECTOR, ".srpTuple__tupleDetails, .tupleNew__TupleContent")
        print(f"Found {len(property_rows)} properties on page {page_count}")
        
        for row in property_rows:
            try:
                property_data = extract_property_data(row)
                data.append(property_data)
                total_properties += 1
            except Exception as e:
                print(f"Error extracting property data: {e}")
        
        # Check if next page exists and navigate to it
        try:
            # Find the next page button by multiple possible selectors
            next_page_selectors = [
                "//a[contains(text(),'Next') or contains(text(),'next')]",
                "//a[contains(@class,'pgdiv')][contains(text(),'Next')]",
                "//div[contains(@class,'pagination')]//a[contains(text(),'Next')]",
                "//a[normalize-space()='Next Page >']"
            ]
            
            next_page_button = None
            for selector in next_page_selectors:
                try:
                    next_page_button = driver.find_element(By.XPATH, selector)
                    break
                except:
                    continue
            
            if not next_page_button:
                print(f"No next page button found. Scraped {page_count} pages in total.")
                break
                
            # Check if "Next" is actually clickable (not disabled)
            if "disabled" in next_page_button.get_attribute("class"):
                print(f"Next page button is disabled. Scraped {page_count} pages in total.")
                break
                
            # Scroll to the next page button
            scroll_to_element(driver, next_page_button)
            
            # Click the next page button
            next_page_button.click()
            print(f"Navigated to page {page_count + 1}")
            
            # Wait for the new page to load
            time.sleep(5)
            wait_for_page_to_load(driver)
            
        except Exception as e:
            print(f"Error navigating to next page: {e}")
            print(f"Completed scraping {page_count} pages.")
            break

except Exception as e:
    print(f"Unexpected error: {e}")
finally:
    print(f"Scraped {total_properties} properties from {page_count} pages.")
    driver.quit()

# ----- CLEANING THE DATA -----

if data:
    df_properties = pd.DataFrame(data)
    
    # Clean and transform the data
    df_properties = (
        df_properties
        .drop_duplicates()
        .apply(lambda col: col.str.strip().str.lower() if isinstance(col, pd.Series) and col.dtype == "object" else col)
        .assign(
            is_starred=lambda df_: df_.name.str.contains("\n", na=False).astype(int),
            name=lambda df_: (
                df_
                .name
                .str.replace("\n[0-9.]+", "", regex=True)
                .str.strip()
                .replace("adroit district s", "adroit district's")
            ),
            location=lambda df_: (
                df_
                .location
                .str.replace("chennai", "", regex=False)
                .str.strip()
                .str.replace(",$", "", regex=True)
                .str.split("in")
                .str[-1]
                .str.strip()
            ),
            price=lambda df_: (
                df_
                .price
                .str.replace("₹", "", regex=False)
                .apply(lambda val: float(val.replace("lac", "").strip()) if isinstance(val, str) and "lac" in val 
                      else float(val.replace("cr", "").strip()) * 100 if isinstance(val, str) and "cr" in val
                      else np.nan)
            ),
            area=lambda df_: (
                df_
                .area
                .str.replace("sqft", "", regex=False)
                .str.strip()
                .str.replace(",", "", regex=False)
                .pipe(lambda ser: pd.to_numeric(ser, errors='coerce'))
            ),
            bhk=lambda df_: (
                df_
                .bhk
                .str.extract(r'(\d+\.?\d*)', expand=False)
                .pipe(lambda ser: pd.to_numeric(ser, errors='coerce'))
            )
        )
        .rename(columns={
            "price": "price_lakhs",
            "area": "area_sqft"
        })
        .reset_index(drop=True)
    )
    
    # Save to Excel
    df_properties.to_excel("chennai-properties-99acres.xlsx", index=False)
    print(f"Data saved to chennai-properties-99acres.xlsx with {len(df_properties)} records.")
else:
    print("No data was collected to save.")

The webpage "India Real Estate Property Site - Buy Sell Rent Properties Portal - 99acres.com" loaded successfully.

Could not click the first search suggestion. Trying alternative approach.
The webpage "Property in Chennai - Real Estate in Chennai" loaded successfully.

Successfully navigated to Chennai properties page.
Budget filter applied successfully.
Could not apply filter: Verified
Still couldn't apply filter: Verified
Could not apply filter: Ready To Move
Still couldn't apply filter: Ready To Move
Could not apply filter: With Photos
Still couldn't apply filter: With Photos
The webpage "Property in Chennai - Real Estate in Chennai" loaded successfully.

All filters applied, proceeding to scrape data.
Processing page 1...
Found 1 properties on page 1
Error navigating to next page: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=135.0.7049.85); For documentation on this error, please visit: https://www.selenium.dev/documentatio

In [11]:
import time
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException

# ----- HELPER FUNCTIONS -----

def wait_for_page_to_load(driver, wait_time=15):
    """Wait for page to load completely"""
    title = driver.title
    wait = WebDriverWait(driver, wait_time)
    try:
        wait.until(lambda d: d.execute_script("return document.readyState") == "complete")
        print(f"The webpage \"{title}\" loaded successfully.\n")
        # Additional wait for any AJAX content to load
        time.sleep(3)
    except:
        print(f"Warning: The webpage \"{title}\" may not have fully loaded.\n")

def scroll_to_element(driver, element):
    """Scroll to make element visible"""
    try:
        driver.execute_script("arguments[0].scrollIntoView({block: 'center', behavior: 'smooth'});", element)
        time.sleep(2)
    except:
        print("Could not scroll to element")

def extract_property_data(row):
    """Extract data from a property row"""
    property_data = {}
    
    # Property name
    try:
        property_data["name"] = row.find_element(By.CSS_SELECTOR, ".tupleNew__headingNrera, .srpTuple__propertyName").text
    except:
        property_data["name"] = np.nan

    # Property location
    try:
        property_data["location"] = row.find_element(By.CSS_SELECTOR, ".tupleNew__propType, .srpTuple__locationName").text
    except:
        property_data["location"] = np.nan

    # Property price
    try:
        property_data["price"] = row.find_element(By.CSS_SELECTOR, ".tupleNew__priceValWrap, .srpTuple__price").text
    except:
        property_data["price"] = np.nan

    # Property area and bhk
    try:
        elements = row.find_elements(By.CSS_SELECTOR, ".tupleNew__area1Type, .srpTuple__dFlex.srpTuple__srpDataWrap span:nth-child(1), .srpTuple__dFlex.srpTuple__srpDataWrap span:nth-child(3)")
        if len(elements) >= 2:
            property_data["area"] = elements[0].text
            property_data["bhk"] = elements[1].text
        else:
            property_data["area"] = elements[0].text if elements else np.nan
            property_data["bhk"] = np.nan
    except:
        property_data["area"] = np.nan
        property_data["bhk"] = np.nan
        
    return property_data

def safe_click(driver, element):
    """Attempt to safely click an element with multiple strategies"""
    try:
        # Try regular click
        element.click()
        return True
    except:
        try:
            # Try JavaScript click
            driver.execute_script("arguments[0].click();", element)
            return True
        except:
            try:
                # Try ActionChains click
                ActionChains(driver).move_to_element(element).click().perform()
                return True
            except:
                return False

def find_element_with_retry(driver, by, value, max_attempts=3):
    """Find an element with retry logic to handle stale references"""
    for attempt in range(max_attempts):
        try:
            return driver.find_element(by, value)
        except StaleElementReferenceException:
            if attempt == max_attempts - 1:
                raise
            time.sleep(1)

def find_elements_with_retry(driver, by, value, max_attempts=3):
    """Find elements with retry logic to handle stale references"""
    for attempt in range(max_attempts):
        try:
            return driver.find_elements(by, value)
        except StaleElementReferenceException:
            if attempt == max_attempts - 1:
                raise
            time.sleep(1)

# ----- MAIN SCRAPING FUNCTION -----

def scrape_99acres():
    # Configure Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--disable-http2")
    chrome_options.add_argument("--incognito")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--ignore-certificate-errors")
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_argument("--disable-popup-blocking")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.112 Safari/537.36"
    )

    # Initialize driver
    driver = webdriver.Chrome(options=chrome_options)
    driver.maximize_window()

    # Create WebDriverWait instance with increased timeout
    wait = WebDriverWait(driver, 20)  # Increased timeout to 20 seconds

    # Track our data
    data = []
    page_count = 0
    total_properties = 0

    try:
        # Access the target webpage
        url = "https://www.99acres.com/"
        driver.get(url)
        wait_for_page_to_load(driver)

        # Search for Chennai properties
        try:
            search_bar = wait.until(EC.presence_of_element_located((By.ID, "keyword2")))
            search_bar.clear()
            search_bar.send_keys("Chennai")
            time.sleep(2)
            
            # Select the first suggestion
            try:
                valid_option = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='autocomplete_drop']//li[1]")))
                safe_click(driver, valid_option)
            except:
                print("Could not click the first search suggestion. Trying alternative approach.")
                search_bar.send_keys(Keys.DOWN)
                search_bar.send_keys(Keys.ENTER)
            
            time.sleep(2)
            
            # Click search button
            search_button = wait.until(EC.element_to_be_clickable((By.ID, "searchform_search_btn")))
            safe_click(driver, search_button)
            wait_for_page_to_load(driver)
            
            print("Successfully navigated to Chennai properties page.")
        except Exception as e:
            print(f"Error during search process: {e}")
        
        # Apply filters
        try:
            # Wait for filters to load
            time.sleep(5)
            
            # Try to apply Budget filter
            try:
                slider = wait.until(EC.presence_of_element_located((By.XPATH, '//*[contains(@id,"budgetLeftFilter_max_node")]')))
                scroll_to_element(driver, slider)
                actions = ActionChains(driver)
                actions.click_and_hold(slider).move_by_offset(-73, 0).release().perform()
                time.sleep(2)
                print("Budget filter applied successfully.")
            except Exception as e:
                print(f"Budget slider adjustment failed: {e}")
            
            # Apply main filters
            filters_to_apply = ["Verified", "Ready To Move", "With Photos", "With Videos"]
            
            for filter_name in filters_to_apply:
                print(f"Attempting to apply {filter_name} filter...")
                
                # Try multiple XPath approaches to find the filter
                xpath_patterns = [
                    f"//div[contains(@class,'srpFilterWrapper')]//span[contains(text(),'{filter_name}')]/parent::*",
                    f"//div[contains(@class,'filterSection')]//span[contains(text(),'{filter_name}')]/parent::*",
                    f"//span[text()='{filter_name}']/parent::*",
                    f"//*[contains(text(),'{filter_name}') and not(ancestor::*[contains(@style,'display: none')])]"
                ]
                
                filter_found = False
                for xpath in xpath_patterns:
                    try:
                        filter_elements = driver.find_elements(By.XPATH, xpath)
                        for element in filter_elements:
                            if element.is_displayed():
                                scroll_to_element(driver, element)
                                if safe_click(driver, element):
                                    filter_found = True
                                    print(f"Applied filter: {filter_name}")
                                    time.sleep(2)
                                    break
                        if filter_found:
                            break
                    except Exception as e:
                        continue
                
                # If filter not found, try to scroll right to see more filters
                if not filter_found:
                    try:
                        right_arrows = driver.find_elements(By.XPATH, "//i[contains(@class,'rightArrow')] | //span[contains(@class,'rightArrow')]")
                        for arrow in right_arrows:
                            if arrow.is_displayed():
                                scroll_to_element(driver, arrow)
                                if safe_click(driver, arrow):
                                    print("Scrolled filter section right")
                                    time.sleep(3)
                                    
                                    # Try filter again after scrolling
                                    for xpath in xpath_patterns:
                                        try:
                                            filter_elements = driver.find_elements(By.XPATH, xpath)
                                            for element in filter_elements:
                                                if element.is_displayed():
                                                    scroll_to_element(driver, element)
                                                    if safe_click(driver, element):
                                                        filter_found = True
                                                        print(f"Applied filter: {filter_name} after scrolling")
                                                        time.sleep(2)
                                                        break
                                            if filter_found:
                                                break
                                        except:
                                            continue
                                    break
                    except Exception as e:
                        print(f"Could not scroll filter section: {e}")
                
                if not filter_found:
                    print(f"Could not apply filter: {filter_name}")
            
            # Wait for filters to apply
            time.sleep(5)
            wait_for_page_to_load(driver)
            
        except Exception as e:
            print(f"Error applying filters: {e}")
        
        # ===== PAGINATION STRATEGY: DIRECT URL MANIPULATION =====
        # Get current URL to use as base for pagination
        base_url = driver.current_url
        
        # Remove any existing page parameters
        if "page=" in base_url:
            base_url = base_url.split("page=")[0]
            if base_url.endswith("&"):
                page_url_template = f"{base_url}page="
            else:
                page_url_template = f"{base_url}&page="
        else:
            if "?" in base_url:
                page_url_template = f"{base_url}&page="
            else:
                page_url_template = f"{base_url}?page="
        
        # Start with page 1
        current_page = 1
        max_pages_to_scrape = 20  # Limit to prevent infinite loops
        consecutive_empty_pages = 0
        
        while current_page <= max_pages_to_scrape and consecutive_empty_pages < 3:
            page_url = f"{page_url_template}{current_page}"
            print(f"Navigating to page {current_page}: {page_url}")
            
            driver.get(page_url)
            wait_for_page_to_load(driver)
            page_count += 1
            
            # Scroll down the page gradually to ensure all content loads
            driver.execute_script("window.scrollTo(0, 0);")  # Start at top
            for i in range(10):  # Scroll in 10 steps
                driver.execute_script(f"window.scrollTo(0, {(i+1) * document_height / 10});")
                time.sleep(0.5)
            
            # Get property listings
            try:
                property_rows = wait.until(EC.presence_of_all_elements_located((
                    By.CSS_SELECTOR, ".srpTuple__tupleDetails, .tupleNew__TupleContent"
                )))
                print(f"Found {len(property_rows)} properties on page {current_page}")
                
                if len(property_rows) == 0:
                    consecutive_empty_pages += 1
                    print(f"Empty page detected. Consecutive empty pages: {consecutive_empty_pages}")
                else:
                    consecutive_empty_pages = 0
                
                # Extract data
                for row in property_rows:
                    try:
                        property_data = extract_property_data(row)
                        data.append(property_data)
                        total_properties += 1
                    except Exception as e:
                        print(f"Error extracting property data: {e}")
                        
            except TimeoutException:
                print(f"No properties found on page {current_page}")
                consecutive_empty_pages += 1
                print(f"Consecutive empty pages: {consecutive_empty_pages}")
                
            # Go to next page
            current_page += 1
            time.sleep(3)  # Wait before loading next page
            
    except Exception as e:
        print(f"Unexpected error: {e}")
    finally:
        print(f"Scraped {total_properties} properties from {page_count} pages.")
        driver.quit()
        return data

# ----- RUN THE SCRAPER -----

data = scrape_99acres()

# ----- CLEANING THE DATA -----

if data:
    df_properties = pd.DataFrame(data)
    
    # Clean and transform the data
    df_properties = (
        df_properties
        .drop_duplicates()
        .apply(lambda col: col.str.strip().str.lower() if isinstance(col, pd.Series) and col.dtype == "object" else col)
    )
    
    # Process each column individually with error handling
    try:
        # Process name column
        if 'name' in df_properties.columns:
            df_properties['is_starred'] = df_properties['name'].str.contains("\n", na=False).astype(int)
            df_properties['name'] = (
                df_properties['name']
                .str.replace("\n[0-9.]+", "", regex=True)
                .str.strip()
                .replace("adroit district s", "adroit district's")
            )
    except Exception as e:
        print(f"Error processing 'name' column: {e}")
        
    try:
        # Process location column
        if 'location' in df_properties.columns:
            df_properties['location'] = (
                df_properties['location']
                .str.replace("chennai", "", regex=False)
                .str.strip()
                .str.replace(",$", "", regex=True)
                .str.split("in")
                .str[-1]
                .str.strip()
            )
    except Exception as e:
        print(f"Error processing 'location' column: {e}")
        
    try:
        # Process price column
        if 'price' in df_properties.columns:
            def process_price(val):
                if not isinstance(val, str):
                    return np.nan
                try:
                    val = val.replace("₹", "").strip()
                    if "lac" in val:
                        return float(val.replace("lac", "").strip())
                    elif "cr" in val:
                        return float(val.replace("cr", "").strip()) * 100
                    else:
                        return np.nan
                except:
                    return np.nan
                    
            df_properties['price_lakhs'] = df_properties['price'].apply(process_price)
            df_properties = df_properties.drop('price', axis=1)
    except Exception as e:
        print(f"Error processing 'price' column: {e}")
        
    try:
        # Process area column
        if 'area' in df_properties.columns:
            df_properties['area_sqft'] = (
                df_properties['area']
                .str.replace("sqft", "", regex=False)
                .str.strip()
                .str.replace(",", "", regex=False)
                .pipe(lambda ser: pd.to_numeric(ser, errors='coerce'))
            )
            df_properties = df_properties.drop('area', axis=1)
    except Exception as e:
        print(f"Error processing 'area' column: {e}")
        
    try:
        # Process bhk column
        if 'bhk' in df_properties.columns:
            df_properties['bhk'] = (
                df_properties['bhk']
                .str.extract(r'(\d+\.?\d*)', expand=False)
                .pipe(lambda ser: pd.to_numeric(ser, errors='coerce'))
            )
    except Exception as e:
        print(f"Error processing 'bhk' column: {e}")
    
    # Reset index and save to Excel
    df_properties = df_properties.reset_index(drop=True)
    df_properties.to_excel("chennai-properties-99acres.xlsx", index=False)
    print(f"Data saved to chennai-properties-99acres.xlsx with {len(df_properties)} records.")
else:
    print("No data was collected to save.")

The webpage "India Real Estate Property Site - Buy Sell Rent Properties Portal - 99acres.com" loaded successfully.

Could not click the first search suggestion. Trying alternative approach.
The webpage "Property in Chennai - Real Estate in Chennai" loaded successfully.

Successfully navigated to Chennai properties page.
Budget filter applied successfully.
Attempting to apply Verified filter...
Applied filter: Verified
Attempting to apply Ready To Move filter...
Applied filter: Ready To Move
Attempting to apply With Photos filter...
Applied filter: With Photos
Attempting to apply With Videos filter...
Scrolled filter section right
Applied filter: With Videos after scrolling
The webpage "Property in Chennai - Real Estate in Chennai" loaded successfully.

Navigating to page 1: https://www.99acres.com/search/property/buy/chennai?city=32&preference=S&area_unit=1&res_com=R&page=1
The webpage "Page 1 - Property in Chennai - Real Estate in Chennai" loaded successfully.

Unexpected error: name 

In [ ]:
{
  "data": {
    "loan_analysis": {
      "__typename": "LoanAnalysis",
      "market": {
        "__typename": "Market",
        "mortgage_data": {
          "__typename": "MortgageData",
          "insurance_rate": 0.003,
          "property_tax_rate": 0.0125,
          "average_rates": [
            {
              "__typename": "Rate",
              "loan_type": {
                "__typename": "LoanType",
                "loan_id": "thirty_year_fix",
                "term": 30,
                "display_name": "30-year fixed",
                "is_va_loan": null,
                "is_fixed": true
              },
              "rate": 0.06976
            },
            {
              "__typename": "Rate",
              "loan_type": {
                "__typename": "LoanType",
                "loan_id": "twenty_year_fix",
                "term": 20,
                "display_name": "20-year fixed",
                "is_va_loan": null,
                "is_fixed": true
              },
              "rate": 0.06788
            },
            {
              "__typename": "Rate",
              "loan_type": {
                "__typename": "LoanType",
                "loan_id": "fifteen_year_fix",
                "term": 15,
                "display_name": "15-year fixed",
                "is_va_loan": null,
                "is_fixed": true
              },
              "rate": 0.06006
            },
            {
              "__typename": "Rate",
              "loan_type": {
                "__typename": "LoanType",
                "loan_id": "ten_year_fix",
                "term": 10,
                "display_name": "10-year fixed",
                "is_va_loan": null,
                "is_fixed": true
              },
              "rate": 0.05907
            },
            {
              "__typename": "Rate",
              "loan_type": {
                "__typename": "LoanType",
                "loan_id": "thirty_year_fha",
                "term": 30,
                "display_name": "30-year fixed FHA",
                "is_va_loan": null,
                "is_fixed": true
              },
              "rate": 0.06288
            },
            {
              "__typename": "Rate",
              "loan_type": {
                "__typename": "LoanType",
                "loan_id": "thirty_year_va",
                "term": 30,
                "display_name": "30-year fixed VA",
                "is_va_loan": true,
                "is_fixed": true
              },
              "rate": 0.0638
            },
            {
              "__typename": "Rate",
              "loan_type": {
                "__typename": "LoanType",
                "loan_id": "five_one_arm",
                "term": 30,
                "display_name": "5-year ARM",
                "is_va_loan": null,
                "is_fixed": false
              },
              "rate": 0.07021
            },
            {
              "__typename": "Rate",
              "loan_type": {
                "__typename": "LoanType",
                "loan_id": "seven_one_arm",
                "term": 30,
                "display_name": "7-year ARM",
                "is_va_loan": null,
                "is_fixed": false
              },
              "rate": 0.07096
            }
          ]
        }
      }
    }
  }
}